Parse whatsapp chat export between alex and yashodan

In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
# from googletrans import Translator
import six
from google.cloud import translate_v2 as translate
import six
from tqdm.notebook import tqdm

tqdm.pandas()

pd.set_option('display.max_colwidth', 800)

# translator = Translator(service_urls=[
#       'translate.google.com'
#     ])

# translator.translate(['עברית'])

In [2]:
# def translate_text(target, text, show=False):
#     """Translates text into the target language.

#     Target must be an ISO 639-1 language code.
#     See https://g.co/cloud/translate/v2/translate-reference#supported_languages
#     """

#     translate_client = translate.Client()

#     if isinstance(text, six.binary_type):
#         text = text.decode("utf-8")

#     # Text can also be a sequence of strings, in which case this method
#     # will return a sequence of results for each text.
#     result = translate_client.translate(text, target_language=target)
#     if show:
#         print(u"Text: {}".format(result["input"]))
#         print(u"Translation: {}".format(result["translatedText"]))
#         print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))
#     return result["translatedText"]

# result = translate_text("en", "שלום")
# result

In [3]:
# df = pd.read_csv('ArielLeventhal.txt', delimiter = "\n")
lines = []
with open("ArielLeventhal.txt") as fd:
    lines = fd.readlines()
    
lines = [line.strip('\n') for line in lines]

In [4]:
df = pd.DataFrame(lines)


In [5]:
df.columns = ['message']

In [6]:
def get_author(x):
    if 'Ariel Leventhal:' in x:
        return 'ariel'
    else:
        return 'shachar'

In [7]:
def get_content(x):
    # return ' '.join(x.split(':')[2:]).strip()
    return ':'.join(x.split(':')[2:]).strip()


In [8]:
def scrub_media(x):
    return x.replace("<Media omitted>", "")

In [9]:
def scrub_urls(x):
    return x if 'http' not in x else  ''

In [10]:
def get_ts(x):
    try:
        return datetime.strptime(x.split(" - ")[0].replace(',',''), '%m/%d/%y %I:%M %p')
    except:
        return None

In [11]:
df['author'] = df['message'].apply(lambda x: get_author(x))

In [12]:
df['content'] = df['message'].apply(lambda x: get_content(x))
# df['message'].loc[8], df['content'].loc[8]
# ':'.join(df['message'].loc[8].split(':')[2:]).strip()

In [13]:
df['content'] = df['content'].apply(lambda x: scrub_media(x))

In [14]:
df['content'] = df['content'].apply(lambda x: scrub_urls(x))

In [15]:
df['content_len'] = df['content'].str.len()

In [16]:
df = df[df['content_len'] > 0]

In [17]:
df[:10]

,message,author,content,content_len
1,"9/3/17, 6:12 PM - shachar: Ping",shachar,Ping,4
2,"9/3/17, 10:21 PM - Ariel Leventhal: Pong",ariel,Pong,4
3,"9/3/17, 10:21 PM - Ariel Leventhal: היה כיף",ariel,היה כיף,7
4,"9/3/17, 10:22 PM - Ariel Leventhal: תפוחי אדמה ואבטיח בתל אביב בקרוב",ariel,תפוחי אדמה ואבטיח בתל אביב בקרוב,32
5,"9/3/17, 10:24 PM - shachar: 😜",shachar,😜,1
6,"9/3/17, 10:24 PM - shachar: מעולה",shachar,מעולה,5
7,"9/3/17, 10:25 PM - shachar: תהנה בניו יורק. עם טמפרטורה נורמלית ו vegan food. ניו יורק זה המקום",shachar,תהנה בניו יורק. עם טמפרטורה נורמלית ו vegan food. ניו יורק זה המקום,67
8,"9/3/17, 10:33 PM - Ariel Leventhal: :)",ariel,:),2
9,"9/3/17, 10:33 PM - Ariel Leventhal: הייתי נשאר לעוד כמה ימי משפחה בכיף",ariel,הייתי נשאר לעוד כמה ימי משפחה בכיף,34
10,"9/4/17, 6:56 PM - shachar: נראה שאני אשאר פוטנציאלית גם קצת להוריקן שאולי יבטל את הטיסות",shachar,נראה שאני אשאר פוטנציאלית גם קצת להוריקן שאולי יבטל את הטיסות,61


In [18]:
# n = 100
# df['content'].iloc[n], translate_text('en', df['content'].iloc[n])
# df['content']

# with open('content.txt', 'a') as f:
#     dfAsString = df['content'].to_string(header=False, index=False)
#     f.write(dfAsString)
def batch_translate_text(
    input_uri="gs://YOUR_BUCKET_ID/path/to/your/file.txt",
    output_uri="gs://YOUR_BUCKET_ID/path/to/save/results/",
    project_id="YOUR_PROJECT_ID",
    timeout=180,
):
    """Translates a batch of texts on GCS and stores the result in a GCS location."""

    client = translate.TranslationServiceClient()

    location = "us-central1"
    # Supported file types: https://cloud.google.com/translate/docs/supported-formats
    gcs_source = {"input_uri": input_uri}

    input_configs_element = {
        "gcs_source": gcs_source,
        "mime_type": "text/plain",  # Can be "text/plain" or "text/html".
    }
    gcs_destination = {"output_uri_prefix": output_uri}
    output_config = {"gcs_destination": gcs_destination}
    parent = f"projects/{project_id}/locations/{location}"

    # Supported language codes: https://cloud.google.com/translate/docs/languages
    operation = client.batch_translate_text(
        request={
            "parent": parent,
            "source_language_code": "en",
            "target_language_codes": ["ja"],  # Up to 10 language codes here.
            "input_configs": [input_configs_element],
            "output_config": output_config,
        }
    )

    print("Waiting for operation to complete...")
    response = operation.result(timeout)

    print("Total Characters: {}".format(response.total_characters))
    print("Translated Characters: {}".format(response.translated_characters))

    
translate_client = translate.Client()
def translate_text(target, text, show=False):
    """Translates text into the target language.

    Target must be an ISO 639-1 language code.
    See https://g.co/cloud/translate/v2/translate-reference#supported_languages
    """

    if isinstance(text, six.binary_type):
        text = text.decode("utf-8")

    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    result = translate_client.translate(text, target_language=target)
    if show:
        print(u"Text: {}".format(result["input"]))
        print(u"Translation: {}".format(result["translatedText"]))
        print(u"Detected source language: {}".format(result["detectedSourceLanguage"]))
    return result["translatedText"]

result = translate_text("en", "שלום")
result

# np.savetxt(r'content.txt', df.content.values, fmt='%s')


# df['content'] = df['content'].progress_apply(lambda c: translate_text('en', c))
# out = 'translations.txt'
# if not os.path.exists(out):
#     with open(out, 'w') as fd:
        
translations = []
for content in tqdm(df['content'], total=len(df['content'])):
    translated = False
    printed = False
    while not translated:
        try:
            translation = translate_text('en', content)
            translated = True
        except:
            if not printed:
                printed = True
                print("Trying:")
                print(content)
    translations.append(translation)
    

  0%|          | 0/21856 [00:00<?, ?it/s]

In [21]:
import pickle 
with open("dump.pkl", "wb") as fd:
    pickle.dump(translations, fd)

In [33]:
df['raw_content'] = df['content']
df['content'] = df['english_content']
df = df.drop(['english_content'], axis=1)


/home/shacharm/.pyenv/versions/3.7.9/envs/ug/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/shacharm/.pyenv/versions/3.7.9/envs/ug/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,message,author,content,content_len,ts,replying_to_author,replying_to,raw_content
1,"9/3/17, 6:12 PM - shachar: Ping",shachar,Pong,4,2017-09-03 18:12:00,NaN,NaN,Pong
2,"9/3/17, 10:21 PM - Ariel Leventhal: Pong",ariel,It was fun,4,2017-09-03 22:21:00,shachar,Ping,It was fun
3,"9/3/17, 10:21 PM - Ariel Leventhal: היה כיף",ariel,Potatoes and watermelon in Tel Aviv soon,7,2017-09-03 22:21:00,ariel,Pong,Potatoes and watermelon in Tel Aviv soon
4,"9/3/17, 10:22 PM - Ariel Leventhal: תפוחי אדמה ואבטיח בתל אביב בקרוב",ariel,😜,32,2017-09-03 22:22:00,ariel,היה כיף,😜
5,"9/3/17, 10:24 PM - shachar: 😜",shachar,Excellent,1,2017-09-03 22:24:00,ariel,תפוחי אדמה ואבטיח בתל אביב בקרוב,Excellent
...,...,...,...,...,...,...,...,...
21848,"8/31/21, 1:58 PM - shachar: שלפני היפנית",shachar,Like puff pastry,12,2021-08-31 13:58:00,ariel,17:30 זמן ישראל?,Like puff pastry
21849,"8/31/21, 1:59 PM - shachar: סי",shachar,Just tastier,2,2021-08-31 13:59:00,shachar,שלפני היפנית,Just tastier
21851,"8/31/21, 4:03 PM - shachar: בחיי שאני לא יודע למה יוצא לו R יווני",shachar,This is going to be a long experiment,37,2021-08-31 16:03:00,shachar,סי,This is going to be a long experiment
21852,"8/31/21, 4:04 PM - shachar: נדבר על זה ומה הלאה",shachar,Hodo Soy Beanery?,19,2021-08-31 16:04:00,shachar,בחיי שאני לא יודע למה יוצא לו R יווני,Hodo Soy Beanery?


In [34]:
df['ts'] = df['message'].apply(lambda x: get_ts(x))
df['ts'] = pd.to_datetime(df['ts'])

In [35]:
    # return datetime.strptime(x.split(" -")[0].replace(',',''), '%d/%m/%y %I:%M %p')
    # datetime.strptime('9/18/17 9:38 PM', '%d/%m/%y %I:%M %p')
    # datetime.strptime('9/18/17 9:38 PM', '%d/%m/%y %I:%M %p')

In [36]:
# df["message"].apply(lambda x: translator.translate(x) if x is not None else '')
# translator.translate


In [37]:
df['replying_to_author'] = df['author'].shift(1)
df['replying_to'] = df['content'].shift(1)

In [40]:
df.head(20)

,message,author,content,content_len,ts,replying_to_author,replying_to,raw_content
1,"9/3/17, 6:12 PM - shachar: Ping",shachar,Pong,4,2017-09-03 18:12:00,NaN,NaN,Pong
2,"9/3/17, 10:21 PM - Ariel Leventhal: Pong",ariel,It was fun,4,2017-09-03 22:21:00,shachar,Pong,It was fun
3,"9/3/17, 10:21 PM - Ariel Leventhal: היה כיף",ariel,Potatoes and watermelon in Tel Aviv soon,7,2017-09-03 22:21:00,ariel,It was fun,Potatoes and watermelon in Tel Aviv soon
4,"9/3/17, 10:22 PM - Ariel Leventhal: תפוחי אדמה ואבטיח בתל אביב בקרוב",ariel,😜,32,2017-09-03 22:22:00,ariel,Potatoes and watermelon in Tel Aviv soon,😜
5,"9/3/17, 10:24 PM - shachar: 😜",shachar,Excellent,1,2017-09-03 22:24:00,ariel,😜,Excellent
6,"9/3/17, 10:24 PM - shachar: מעולה",shachar,Enjoy New York. With normal temperature and vegan food. New York is the place,5,2017-09-03 22:24:00,shachar,Excellent,Enjoy New York. With normal temperature and vegan food. New York is the place
7,"9/3/17, 10:25 PM - shachar: תהנה בניו יורק. עם טמפרטורה נורמלית ו vegan food. ניו יורק זה המקום",shachar,:),67,2017-09-03 22:25:00,shachar,Enjoy New York. With normal temperature and vegan food. New York is the place,:)
8,"9/3/17, 10:33 PM - Ariel Leventhal: :)",ariel,I would stay for a few more family days for fun,2,2017-09-03 22:33:00,shachar,:),I would stay for a few more family days for fun
9,"9/3/17, 10:33 PM - Ariel Leventhal: הייתי נשאר לעוד כמה ימי משפחה בכיף",ariel,It looks like I&#39;ll potentially also stay a bit in a hurricane that might cancel the flights,34,2017-09-03 22:33:00,ariel,I would stay for a few more family days for fun,It looks like I&#39;ll potentially also stay a bit in a hurricane that might cancel the flights
10,"9/4/17, 6:56 PM - shachar: נראה שאני אשאר פוטנציאלית גם קצת להוריקן שאולי יבטל את הטיסות",shachar,:),61,2017-09-04 18:56:00,ariel,It looks like I&#39;ll potentially also stay a bit in a hurricane that might cancel the flights,:)


In [41]:
df['is_reply'] = df['replying_to_author'] != df['author']

In [42]:
df = df[df['is_reply'] == 1]

In [44]:
!tail -20 'ArielLeventhal.txt'

12/18/21, 5:55 PM - shachar: עד גיל שנה לראות מה שרוצים כי הם לא מבינים
12/18/21, 5:55 PM - shachar: מגיל 3 רואים כבר מיניונים ושרק
12/18/21, 7:09 PM - shachar: אבל בוא. הילד זה רק תירוץ.
12/18/21, 7:12 PM - shachar: <Media omitted>
12/19/21, 1:51 AM - Ariel Leventhal: <Media omitted>
12/19/21, 1:51 AM - Ariel Leventhal: <Media omitted>
12/19/21, 1:51 AM - Ariel Leventhal: רק טעים יותר
12/19/21, 1:58 AM - shachar: רגע מה
12/19/21, 1:58 AM - shachar: מה טעים יותר
12/19/21, 2:05 AM - Ariel Leventhal: יובה
12/19/21, 2:05 AM - Ariel Leventhal: באפיה
12/19/21, 2:05 AM - Ariel Leventhal: תופח
12/19/21, 2:05 AM - Ariel Leventhal: כמו בצק עלים
12/19/21, 2:05 AM - Ariel Leventhal: רק טעים יותר
12/19/21, 2:08 AM - shachar: 😱
12/19/21, 2:17 AM - shachar: זה הולך להיות ניסוי ארוך
12/19/21, 2:19 AM - shachar: Hodo Soy Beanery?
12/19/21, 2:22 AM - Ariel Leventhal: יפ
12/19/21, 2:22 AM - Ariel Leventhal: היחיד בארצות הברית
12/19/21, 2:26 AM - shachar: 🙄


In [45]:
df.head(20)

,message,author,content,content_len,ts,replying_to_author,replying_to,raw_content,is_reply
1,"9/3/17, 6:12 PM - shachar: Ping",shachar,Pong,4,2017-09-03 18:12:00,NaN,NaN,Pong,True
2,"9/3/17, 10:21 PM - Ariel Leventhal: Pong",ariel,It was fun,4,2017-09-03 22:21:00,shachar,Pong,It was fun,True
5,"9/3/17, 10:24 PM - shachar: 😜",shachar,Excellent,1,2017-09-03 22:24:00,ariel,😜,Excellent,True
8,"9/3/17, 10:33 PM - Ariel Leventhal: :)",ariel,I would stay for a few more family days for fun,2,2017-09-03 22:33:00,shachar,:),I would stay for a few more family days for fun,True
10,"9/4/17, 6:56 PM - shachar: נראה שאני אשאר פוטנציאלית גם קצת להוריקן שאולי יבטל את הטיסות",shachar,:),61,2017-09-04 18:56:00,ariel,It looks like I&#39;ll potentially also stay a bit in a hurricane that might cancel the flights,:),True
11,"9/4/17, 7:12 PM - Ariel Leventhal: :)",ariel,The hurricane sounds cool,2,2017-09-04 19:12:00,shachar,:),The hurricane sounds cool,True
13,"9/4/17, 7:13 PM - shachar: יהיה מעניין",shachar,:),11,2017-09-04 19:13:00,ariel,Will be interesting,:),True
15,"9/4/17, 7:13 PM - Ariel Leventhal: :)",ariel,flap,2,2017-09-04 19:13:00,shachar,:),flap,True
19,"9/8/17, 2:01 AM - shachar: נראה לי.",shachar,I booked a flight tonight,8,2017-09-08 02:01:00,ariel,In line for the flight,I booked a flight tonight,True
23,"9/8/17, 2:02 AM - Ariel Leventhal: נסיעה טובה, בן דוד :)",ariel,Great idea,21,2017-09-08 02:02:00,shachar,See you soon,Great idea,True


In [46]:
df_a = df[df['author'] == 'ariel'][['content','replying_to']]
df_a.columns = ['completion', 'prompt']
df_a.to_csv('data_ariel.csv', index=False)
data_a = list(df_a.to_dict(orient='index').values())

In [47]:
df_b = df[df['author'] == 'shachar'][['content','replying_to']]
df_b.columns = ['completion', 'prompt']
df_b.to_csv('data_shachar.csv', index=False)
data_b = list(df_b.to_dict(orient='index').values())

In [48]:
with open('data_ariel.jsonl', 'w') as f:
    for item in data_a:
        f.write("%s\n" % item)

In [49]:
with open('data_shachar.jsonl', 'w') as f:
    for item in data_b:
        f.write("%s\n" % item)